In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain.llms import Ollama
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [8]:
import pandas as pd
books = pd.read_csv('books_cleaned.csv')

In [9]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [10]:
books['tagged_description']

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [11]:
books['tagged_description'].to_csv('tagged_description.txt', sep = '\n', index = False, header = False )

In [17]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()

text_splitter = CharacterTextSplitter(
    chunk_size=500,        # ✅ Valid chunk size
    chunk_overlap=0,
    separator="\n"
)

documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 500
Created a chunk of size 1214, which is longer than the specified 500
Created a chunk of size 960, which is longer than the specified 500
Created a chunk of size 843, which is longer than the specified 500
Created a chunk of size 881, which is longer than the specified 500
Created a chunk of size 1088, which is longer than the specified 500
Created a chunk of size 1189, which is longer than the specified 500
Created a chunk of size 513, which is longer than the specified 500
Created a chunk of size 752, which is longer than the specified 500
Created a chunk of size 728, which is longer than the specified 500
Created a chunk of size 721, which is longer than the specified 500
Created a chunk of size 1267, which is longer than the specified 500
Created a chunk of size 681, which is longer than the specified 500
Created a chunk of size 553, which is longer than the specified 500
Created a chunk of size 521, which is longe

In [18]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [19]:
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

db_books = Chroma.from_documents(
    documents,
    embedding=embedding
)

C:\Users\j_san\AppData\Local\Temp\ipykernel_12984\582706991.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [20]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(metadata={'source': 'tagged_description.txt'}, page_content="9780786808397 Introduce your baby to birds, cats, dogs, and babies through fine art, illu

In [21]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


Now we want a function specifically to provide similar books based on query

In [22]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10  #number of results basically
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = top_k)

    books_list = []
    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [24]:
retrieve_semantic_recommendations("A book about cooking")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
726,9780141010373,0141010371,Jamie's Kitchen,Jamie Oliver,Cookery,http://books.google.com/books/content?id=WG3SN...,The No 1 bestseller I love this book. It's the...,2004.0,3.89,336.0,4012.0,Jamie's Kitchen,9780141010373 The No 1 bestseller I love this ...
1886,9780393011838,0393011836,What Einstein Told His Cook,Robert L. Wolke;Marlene Parrish,Cooking,http://books.google.com/books/content?id=6Hbbf...,The chemistry professor columnist for the Wash...,2002.0,3.85,320.0,3058.0,What Einstein Told His Cook: Kitchen Science E...,9780393011838 The chemistry professor columnis...
3678,9780767909266,0767909267,The Minimalist Cooks at Home,Mark Bittman,Cooking,http://books.google.com/books/content?id=F0Z6P...,"In a revised cookbook, the author of the award...",2002.0,3.91,240.0,261.0,The Minimalist Cooks at Home: Recipes That Giv...,"9780767909266 In a revised cookbook, the autho..."
4341,9781400040353,1400040353,Lidia's Family Table,Lidia Matticchio Bastianich;David Nussbaum,Cooking,http://books.google.com/books/content?id=ZQNwD...,A guide to family cookery features more than t...,2004.0,4.22,448.0,521.0,Lidia's Family Table,9781400040353 A guide to family cookery featur...
4349,9781400043460,1400043468,My Life in France,Julia Child;Alex Prud'homme,Biography & Autobiography,http://books.google.com/books/content?id=KmuMD...,A memoir begun just months before Child's deat...,2006.0,4.17,302.0,67593.0,My Life in France,9781400043460 A memoir begun just months befor...
4363,9781400052387,1400052386,Tyler's Ultimate,Tyler Florence,Cooking,http://books.google.com/books/content?id=cGxOH...,The popular Food Network chef presents a selec...,2006.0,4.10,256.0,3586.0,Tyler's Ultimate: Brilliant Simple Food to Mak...,9781400052387 The popular Food Network chef pr...
4428,9781401301941,1401301940,Jamie's Dinners,Jamie Oliver,Cooking,http://books.google.com/books/content?id=gMDlA...,Cooking sensation Jamie Oliver returns with a ...,2004.0,4.07,336.0,4248.0,Jamie's Dinners: The Essential Family Cookbook,9781401301941 Cooking sensation Jamie Oliver r...
4429,9781401301958,1401301959,Jamie's Italy,Jamie Oliver,Cooking,http://books.google.com/books/content?id=e1sYA...,Bestselling author Jamie Oliver finally devote...,2006.0,4.00,320.0,7463.0,Jamie's Italy,9781401301958 Bestselling author Jamie Oliver ...
4669,9781568984957,1568984952,"Yours in Food, John Baldessari",John Baldessari;Paul Auster,Art,http://books.google.com/books/content?id=FtnNd...,"In John Baldessari's new book, Yours in Food, ...",2004.0,4.00,144.0,19.0,"Yours in Food, John Baldessari","9781568984957 In John Baldessari's new book, Y..."
5176,9781933615097,1933615095,The Best of America's Test Kitchen 2007,America's Test Kitchen,Cooking,http://books.google.com/books/content?id=tQ97A...,Presents nearly one thousand recipes--from app...,2006.0,4.34,312.0,185.0,The Best of America's Test Kitchen 2007: The Y...,9781933615097 Presents nearly one thousand rec...
